## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
import sqlite3
import os
import tensorflow as tf

# Import and read the diabetes_data.csv.
#diabetes_df = pd.read_csv("Resources/diabetes_data.csv")

#Define the paths
resources_folder = os.path.join(os.getcwd(), 'resources')
csv_file_path = os.path.join(resources_folder, 'diabetes_data.csv')
database_path = os.path.join(os.getcwd(), 'my_database2.db')

#Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

#Connect to the SQLite database (or create it)
connection = sqlite3.connect(database_path)

#Save the DataFrame to the SQLite database
df.to_sql('diabetes_data', connection, if_exists='replace', index=False)

#Commit and close the connection
connection.commit()
connection.close()

#Verify the data by querying the database
connection = sqlite3.connect(database_path)
diabetes_df = pd.read_sql('SELECT * FROM diabetes_data', connection)
diabetes_BMI = pd.read_sql('SELECT BMI FROM diabetes_data', connection)
diabetes_df.head()
#diabetes_BMI.head(4)

# Drop the column smoking_history 
diabetes_df.drop('smoking_history', axis=1, inplace=True)

# Remove all rows where gender is 'Other'
diabetes_df = diabetes_df[diabetes_df['gender'] != 'Other']

# Reset the index after removing rows
diabetes_df.reset_index(drop=True, inplace=True)
diabetes_df.head()

,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,25.19,6.6,140,0
1,Female,54.0,0,0,27.32,6.6,80,0
2,Male,28.0,0,0,27.32,5.7,158,0
3,Female,36.0,0,0,23.45,5.0,155,0
4,Male,76.0,1,1,20.14,4.8,155,0


In [2]:
# Determine the number of unique values in each column.
diabetes_df.nunique()

gender                    2
age                     102
hypertension              2
heart_disease             2
bmi                    4247
HbA1c_level              18
blood_glucose_level      18
diabetes                  2
dtype: int64

In [3]:
# Convert gender values column to numerical values
diabetes_df = diabetes_df.replace({'gender': {'Female': 0, 'Male': 1}})
diabetes_df.head()

,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0,80.0,0,1,25.19,6.6,140,0
1,0,54.0,0,0,27.32,6.6,80,0
2,1,28.0,0,0,27.32,5.7,158,0
3,0,36.0,0,0,23.45,5.0,155,0
4,1,76.0,1,1,20.14,4.8,155,0


In [4]:
# Split our preprocessed data into our features and target arrays
y = diabetes_df['diabetes'].values
X = diabetes_df.drop('diabetes', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Compile, Train and Evaluate the Model

In [5]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

C:\Users\Victor Doan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │              64 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              45 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 115 (460.00 B)

 Trainable params: 115 (460.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])



In [7]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 3s 651us/step - accuracy: 0.8988 - loss: 0.3952
Epoch 2/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 646us/step - accuracy: 0.9263 - loss: 0.2388
Epoch 3/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 683us/step - accuracy: 0.9313 - loss: 0.2051
Epoch 4/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 634us/step - accuracy: 0.9369 - loss: 0.1826
Epoch 5/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 637us/step - accuracy: 0.9416 - loss: 0.1660
Epoch 6/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 682us/step - accuracy: 0.9472 - loss: 0.1515
Epoch 7/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 716us/step - accuracy: 0.9511 - loss: 0.1426
Epoch 8/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 725us/step - accuracy: 0.9506 - loss: 0.1400
Epoch 9/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 712us/step - accuracy: 0.9527 - loss: 0.1343
Epoch 10/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 709us/step - accuracy: 0.9543 - loss: 0.1299
Epoch 11/100
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 667us/step - accuracy: 0.9531 - loss: 0.12

In [8]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

782/782 - 1s - 688us/step - accuracy: 0.9638 - loss: 0.1064
Loss: 0.1063760444521904, Accuracy: 0.9637942314147949


In [9]:
# Export our model to HDF5 file
nn.save('diabetes_prediction_model.h5')